In [28]:
#load libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
from datetime import datetime, timedelta
from copy import deepcopy
import re, os, ast

In [3]:
raw_data = pd.read_csv("/Users/albk/Documents/Code/Real_Projects/basketball_league_webapp/data/Carleton_at Guelph * ^_2023-11-05.csv")

In [4]:
player_event_df = raw_data.copy()

In [5]:
player_event_df['V-event'] = player_event_df['V-event'].astype(str)

In [6]:
#clean data
players_dict = ast.literal_eval(player_event_df.iloc[0]["Home"])
players_list = players_dict["starters"].copy()
players_list.extend(players_dict["reserves"])
players_list.remove("Team")
event_list = ['made layup','missed layup','Assist','Turnover','defensive rebound','enters the game','goes to the bench','missed 3-pt. jump shot','Foul','Steal','made free throw','missed free throw','made jump shot','made 3-pt. jump shot','missed jump shot','offensive rebound']

In [7]:
players_list

['Marjok Okado',
 'Emmanuel Ugbah',
 'Xavier Spencer',
 'Augustas Brazdeikis',
 'Noah Simpson',
 'Ben Riley',
 'Aubrey Dorey-Havens',
 'To Randriasalama',
 'Wazir Latiff',
 'Emanuel Milon',
 'Reginald Jean Seraphin',
 'Daniel Smith']

In [8]:
def make_swap_uppernames(ls):
    formatted_players = []
    for player in ls:
        name_parts = player.split(" ")
        first_name = " ".join(name_parts[:-1])
        last_name = name_parts[-1]
        formatted_name = last_name.upper() + "," + first_name.upper()
        formatted_players.append(formatted_name)

    return formatted_players

In [9]:
players_list = make_swap_uppernames(players_list)
starters = make_swap_uppernames(players_dict["starters"])

### Simply building a dataframe from main one and getting kinds of event with respect to their player

In [10]:
pattern = "([A-Z]+\W*[A-Z]+,[A-Z]+\W*[A-Z]+)"
player_event_df["V-event"] = player_event_df["V-event"].fillna("No Event")
for index, row in player_event_df.iterrows():
    player = re.search(pattern, row["V-event"])
    if player:
        player = player[0].strip()
        player_event_df.loc[index, "player"] = player
    
    for event in event_list:
        if event in row["V-event"]:
            player_event_df.loc[index, "event"] = event

In [11]:
player_event_df.head(n=20)

,Unnamed: 0,Time,Home,H-event,Score,V-event,Visitor,player,event
0,0,Quarter 1,"{'starters': ['Marjok Okado', 'Emmanuel Ugbah', 'Xavier Spencer', 'Augustas Brazdeikis', 'Noah Simpson'], 'reserves': ['Ben Riley', 'Aubrey Dorey-Havens', 'To Randriasalama', 'Wazir Latiff', 'Emanuel Milon', 'Reginald Jean Seraphin', 'Daniel Smith', 'Team']}",Quarter 1,Quarter 1,Quarter 1,Quarter 1,NaN,NaN
1,1,09:44,Guelph,"WALKER,SHAATHAN missed layup",0 - 0,nan,NaN,NaN,NaN
2,2,09:44,NaN,NaN,0 - 0,"OKADO,MARJOK defensive rebound",Carleton,"OKADO,MARJOK",defensive rebound
3,3,09:29,NaN,NaN,0 - 2,"SIMPSON,NOAH made layup",Carleton,"SIMPSON,NOAH",made layup
4,4,09:29,NaN,NaN,0 - 2,"Assist by BRAZDEIKIS,AUGUSTAS",Carleton,"BRAZDEIKIS,AUGUSTAS",Assist
5,5,08:14,NaN,NaN,0 - 2,"BRAZDEIKIS,AUGUSTAS missed jump shot",Carleton,"BRAZDEIKIS,AUGUSTAS",missed jump shot
6,6,08:14,Guelph,"MINGO,DEZAYNE defensive rebound",0 - 2,nan,NaN,NaN,NaN
7,7,08:06,Guelph,"OLIVER,MATHEW missed 3-pt. jump shot",0 - 2,nan,NaN,NaN,NaN
8,8,08:06,Guelph,"MINGO,DEZAYNE offensive rebound",0 - 2,nan,NaN,NaN,NaN
9,9,07:58,NaN,NaN,0 - 2,"Foul by OKADO,MARJOK",Carleton,"OKADO,MARJOK",Foul


### Calculating in-game time of each player on each quarter and events count for each player also

In [60]:
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
# dependencies
player_event_df["Time"] = pd.to_datetime(player_event_df["Time"], format="%M:%S", errors="coerce")
under5min_df = player_event_df.loc[player_event_df["Time"] < datetime.strptime("05:00", "%M:%S")]

quarter_indices5min = list(reversed(under5min_df["Time"].nlargest(4).index))
quarter_indices = list(player_event_df[player_event_df['Score'].str.contains('Quarter')].index)
quarter_indices.append(len(player_event_df) - 1)
quarter_dict = {"player":players_list, "time1":[], "score1":[], 
                "time2":[], "score2":[], "time3":[], "score3":[],
                "time4":[], "score4":[]}

v = list(quarter_dict.keys())
v.remove("player")
reorder_ls = ["lineup"] + v
lineup_quarter_dict = {key:[] for key in reorder_ls}

# Last 5 minutes statistics of quarters 2 and 4
quarter_dict5min = {"player":players_list, "time2":[],
                    "score2":[], "time4":[], "score4":[]}

quarter = 1
in_lineup = []
starter_lineup = starters.copy()

not_changed_list = ["not_changed" for n in range(len(players_list))]
event_num_dict = {k:[] for k in ["player"] + event_list}
event_num_dict5min = {k:[] for k in ["player"] + [e + "2" for e in event_list] + [e + "4" for e in event_list]}
time_dict = {"player":players_list, 
            "seconds":list(np.zeros(len(players_list))),
            "points_conceded":list(np.zeros(len(players_list))),
            "timecache":not_changed_list.copy(),
            "scorecache":not_changed_list.copy(),
            "seconds5min":list(np.zeros(len(players_list))),
            "points_conceded5min":list(np.zeros(len(players_list))),
            "timecache5min":not_changed_list.copy(),
            "scorecache5min":not_changed_list.copy(),}
             
lineup_time_dict = {k:[] for k in list(time_dict.keys())}
lineup_time_dict["lineup"] = lineup_time_dict.pop("player")
lineup_event_dict = {"lineup":[]}


for ind, row in player_event_df.iterrows():
    # 5min checking needs these constants
    cur_time = deepcopy(row["Time"])
    threshold_time = datetime.strptime("05:00", "%M:%S")
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
    # calculating in-game time of each player
    # -------------------------------------------
    # each quarter end calculations
    if ind in quarter_indices:
        if ind != 0:
            for player in in_lineup:
                player_ind = time_dict["player"].index(player)
                cached_time = time_dict["timecache"][player_ind]
                cached_score = time_dict["scorecache"][player_ind]
                if cached_time == "not_changed":
                    enter_time = datetime.strptime("10:00", "%M:%S")
                    enter_score_index = quarter_indices[quarter - 1] + 1
                    enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
                else:
                    enter_time = cached_time
                    enter_score = cached_score
                
                seconds = enter_time - datetime.strptime("00:00", "%M:%S")
                seconds = seconds.total_seconds()

                points = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0]) - enter_score

                time_dict["seconds"][player_ind] += seconds
                time_dict["points_conceded"][player_ind] += points
            
            # ---------------------
            # lineup quarter calculations
            lineup_cached_score = lineup_time_dict["scorecache"][-1]
            if lineup_time_dict["timecache"][-1] == "not_changed":
                lineup_time_dict["seconds"] += 600
                enter_score_index = quarter_indices[quarter - 1] + 1
                enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
            else:
                cached_time = lineup_time_dict["timecache"][-1]
                enter_time = cached_time
                seconds = enter_time - datetime.strptime("00:00", "%M:%S")
                seconds = seconds.total_seconds()
                lineup_time_dict["seconds"][-1] += seconds
                enter_score = int(lineup_cached_score.split("-")[0])
            
            points = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0]) - enter_score

            lineup_time_dict["seconds"][-1] += seconds
            lineup_time_dict["points_conceded"][-1] += points
            # ---------------------

            quarter_dict["player"] = time_dict["player"]
            quarter_dict[f"time{quarter}"] = time_dict["seconds"]
            quarter_dict[f"score{quarter}"] = time_dict["points_conceded"]
            if quarter in (2, 4):
                quarter_dict5min["player"] = time_dict["player"]
                quarter_dict5min[f"time{quarter}"] = time_dict["seconds5min"]
                quarter_dict5min[f"score{quarter}"] = time_dict["points_conceded5min"]

            for key in list(lineup_quarter_dict.keys()):
                if key != "lineup":
                    if int(key[-1]) == quarter:
                        if "time" in key:
                            lineup_quarter_dict[f"time{quarter}"].extend(lineup_time_dict["seconds"])
                        else:    
                            lineup_quarter_dict[f"score{quarter}"].extend(lineup_time_dict["points_conceded"])
                    else:
                        length = len(lineup_time_dict["lineup"])
                        zero_list = list(np.zeros(length))
                        lineup_quarter_dict[key].extend(zero_list)
                else:
                    lineup_quarter_dict[key].extend(lineup_time_dict["lineup"])

            quarter += 1
            time_dict["timecache"] = not_changed_list.copy()
            time_dict["scorecache"] = not_changed_list.copy()
            time_dict["timecache5min"] = not_changed_list.copy()
            time_dict["scorecache5min"] = not_changed_list.copy()
            time_dict["seconds"] = list(np.zeros(len(players_list)))
            time_dict["points_conceded"] = list(np.zeros(len(players_list)))
            time_dict["seconds5min"] = list(np.zeros(len(players_list)))
            time_dict["points_conceded5min"] = list(np.zeros(len(players_list)))
            lineup_time_dict = {key:[] for key in lineup_time_dict}

        in_lineup = starter_lineup.copy()
        lineup_time_dict["lineup"].append(sorted(in_lineup.copy()))
        for key in lineup_time_dict:
            if key != "lineup":
                if "cache" in key:
                    lineup_time_dict[key].append("not_changed")
                else:
                    lineup_time_dict[key].append(0)

        for key in list(lineup_event_dict.keys()):
                if key == "lineup":
                    lineup_event_dict[key].append(sorted(in_lineup))
                else:
                    lineup_event_dict[key].append(0)
        continue
    # -------------------------------------------
    # iterating rows calculation
    if not pd.isna(row["player"]) and row["player"] != np.nan and not pd.isna(row["event"]) and row["event"] != np.nan:
        player_index = time_dict["player"].index(row["player"])
        cached_time = time_dict["timecache"][player_index]
        cached_score = time_dict["scorecache"][player_index]
        if "goes to the bench" in row["event"]:
            in_lineup.remove(row["player"])
            if cached_time == "not_changed":
                enter_time = datetime.strptime("10:00", "%M:%S")
                enter_score_index = quarter_indices[quarter - 1] + 1
                enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
            else:
                enter_time = cached_time
                enter_score = cached_score
            

            seconds = enter_time - row["Time"]
            seconds = seconds.total_seconds()

            points = int(row["Score"].split("-")[0]) - enter_score


            time_dict["seconds"][player_index] += seconds
            time_dict["points_conceded"][player_index] += points

            ## if player goes to bench in last 5min of quarters 2 an 4 
            ## (considering not to exceed to to much memory and calculate them when needed)
            if cur_time < threshold_time and quarter in (2, 4):
                cached_time5min = time_dict["timecache5min"][player_index]
                cached_score5min = time_dict["scorecache5min"][player_index]
                if cached_time5min == "not_changed":
                    enter_time5min = threshold_time
                    enter_score_index5min = quarter_indices5min[quarter - 1] + 1
                    enter_score5min = int(player_event_df.iloc[enter_score_index5min]["Score"].split("-")[0])
                else:
                    if cached_time5min > threshold_time:
                        enter_time5min = threshold_time
                        enter_score_index5min = quarter_indices5min[quarter - 1] + 1
                        enter_score5min = int(player_event_df.iloc[enter_score_index5min]["Score"].split("-")[0])
                    else:
                        enter_time5min = cached_time5min
                        enter_score5min = cached_score5min
                
                seconds5min = enter_time5min - row["Time"]
                seconds5min = seconds5min.total_seconds()
                points5min = int(row["Score"].split("-")[0]) - enter_score5min
                time_dict["seconds5min"][player_index] += seconds5min
                time_dict["points_conceded5min"][player_index] += points5min

        elif "enters the game" in row["event"]:
            in_lineup.append(row["player"])
            time_dict["timecache"][player_index] = row["Time"]
            time_dict["scorecache"][player_index] = int(row["Score"].split("-")[0])
            
            ## time and score cache for under 5 min assessment would be catched if the the time is below 05:00 
            ## else i would not change it and it would be the initial list that i made before main loop for it
            if cur_time < threshold_time:
                time_dict["timecache5min"][player_index] = row["Time"]
                time_dict["scorecache5min"][player_index] = int(row["Score"].split("-")[0])
        
        # ---------------------
        # lineup iterating rows calculation
        last_lineup = lineup_time_dict["lineup"][-1]
        if sorted(in_lineup) != sorted(last_lineup) and len(in_lineup) == 5:
            cached_time = lineup_time_dict["timecache"][-1]
            if lineup_time_dict["timecache"][-1] == "not_changed":
                enter_time = datetime.strptime("10:00", "%M:%S")
                enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
            else:
                enter_time = cached_time
                enter_score = int(lineup_time_dict["scorecache"][-1].split("-")[0])

            seconds = enter_time - row["Time"]
            seconds = seconds.total_seconds()

            curr_score = int(row["Score"].split("-")[0])
            points = curr_score - enter_score

            lineup_time_dict["seconds"][-1] += seconds
            lineup_time_dict["points_conceded"][-1] += points

            lineup_time_dict["lineup"].append(sorted(in_lineup.copy()))
            for k in lineup_time_dict:
                if k != "lineup":
                    if "cache" in k:
                        lineup_time_dict[k].append("not_changed")
                    else:
                        lineup_time_dict[k].append(0)

            lineup_time_dict["timecache"].append(row["Time"])
            lineup_time_dict["scorecache"].append(row["Score"])
        # ---------------------
    # -------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
        # # filling event_num_dict for meaesuring how many times each event occured
        if row["player"] not in event_num_dict["player"]:
            event_num_dict["player"].append(row["player"])
            for key in event_num_dict:
                if key != "player":
                    event_num_dict[key].append(0)
        
        player_index = event_num_dict["player"].index(row["player"])
        event_num_dict[row["event"]][player_index] += 1

        if cur_time < threshold_time and quarter in (2, 4):
            if row["player"] not in event_num_dict5min["player"]:
                event_num_dict5min["player"].append(row["player"])
                for key in event_num_dict5min:
                    if key != "player":
                        event_num_dict5min[key].append(0)
            
            player_index = event_num_dict5min["player"].index(row["player"])
            event_num_dict5min[row["event"] + str(quarter)][player_index] += 1

        # ---------------------
        # lineup event calculations
        if len(lineup_event_dict["lineup"]) == 0:
            lineup_event_dict["lineup"].append(sorted(in_lineup))

        last_lineup = lineup_event_dict["lineup"][-1]
        new_lineup = sorted(in_lineup.copy())
        if row["event"] not in lineup_event_dict:
            lineup_event_dict[row["event"]] = list(np.zeros(len(lineup_event_dict["lineup"])))
        
        if new_lineup == last_lineup:
            lineup_event_dict[row["event"]][-1] += 1
        elif new_lineup != last_lineup and len(new_lineup) == 5:
            for key in list(lineup_event_dict.keys()):
                if key == "lineup":
                    lineup_event_dict[key].append(new_lineup)
                else:
                    lineup_event_dict[key].append(0)
        # ---------------------
# ------------------------------------------------------------------------------------------------------------------------------------------------------------

time_score_df = pd.DataFrame(quarter_dict)
time_score_df5min = pd.DataFrame(quarter_dict5min)
lineup_time_score_df = pd.DataFrame(lineup_quarter_dict)
event_num_df = pd.DataFrame(event_num_dict)
event_num_df5min = pd.DataFrame(event_num_dict5min)
lineup_event_df = pd.DataFrame(lineup_event_dict)

time_columns = []
for i in range(1, 5):
    for sub in ["time", "score"]:
        time_columns.append((f"quarter{i}", sub))

event_columns = []
for i in [2, 4]:
    for sub in event_list:
        event_columns.append((f"quarter{i}", sub))

time_score_df.columns = pd.MultiIndex.from_tuples([("player", "player")] + time_columns)
time_score_df5min.columns = pd.MultiIndex.from_tuples([("player", "player")] + [(k, v) for k, v in time_columns if k[-1] not in ("1", "3")])
lineup_time_score_df.columns = pd.MultiIndex.from_tuples([("lineup", "lineup")] + time_columns)
event_num_df5min.columns = pd.MultiIndex.from_tuples([("player", "player")] + event_columns)

time_sum_ls = [(f"quarter{i}", "time") for i in range(1, 5)]
score_sum_ls = [(f"quarter{i}", "score") for i in range(1, 5)]
time_score_df[("total", "time")] = time_score_df[time_sum_ls].sum(axis=1)
time_score_df[("total", "score")] = time_score_df[score_sum_ls].sum(axis=1)
lineup_time_score_df[("total", "time")] = lineup_time_score_df[time_sum_ls].sum(axis=1)
lineup_time_score_df[("total", "score")] = lineup_time_score_df[score_sum_ls].sum(axis=1)

In [62]:
time_score_df

player quarter1       quarter2       quarter3        \
                    player     time score     time score     time score   
0             OKADO,MARJOK    252.0   7.0    265.0   7.0    489.0  21.0   
1           UGBAH,EMMANUEL    411.0   9.0    242.0   8.0    338.0  12.0   
2           SPENCER,XAVIER    430.0  14.0    280.0  16.0    393.0  21.0   
3      BRAZDEIKIS,AUGUSTAS    498.0  13.0    439.0  16.0    374.0  19.0   
4             SIMPSON,NOAH    377.0  13.0    300.0  13.0    227.0  10.0   
5                RILEY,BEN    189.0   7.0     63.0   2.0      0.0   0.0   
6      DOREY-HAVENS,AUBREY    170.0   2.0    361.0  16.0    207.0   2.0   
7         RANDRIASALAMA,TO    150.0   6.0    105.0   2.0    262.0  11.0   
8             LATIFF,WAZIR    198.0   3.0    316.0  19.0    111.0   2.0   
9            MILON,EMANUEL    223.0   3.0    192.0   8.0    373.0  13.0   
10  SERAPHIN,REGINALD JEAN    102.0   3.0    155.0  10.0    226.0   4.0   
11            SMITH,DANIEL      0.0   0.0    173.0   8.0      0.0   0.0   

   quarter4         total        
       time score    time score  
0     559.0  16.0  1565.0  51.0  
1     192.0   8.0  1183.0  37.0  
2     355.0  14.0  1458.0  65.0  
3     408.0  11.0  1719.0  59.0  
4     395.0  11.0  1299.0  47.0  
5       0.0   0.0   252.0   9.0  
6     429.0  16.0  1167.0  36.0  
7       0.0   0.0   517.0  19.0  
8     457.0  11.0  1082.0  35.0  
9     205.0   8.0   993.0  32.0  
10      0.0   0.0   483.0  17.0  
11      0.0   0.0   173.0   8.0

In [63]:
time_score_df5min

player quarter2       quarter4      
                    player     time score     time score
0             OKADO,MARJOK      0.0   0.0    169.0  22.0
1           UGBAH,EMMANUEL    114.0   5.0      1.0   0.0
2           SPENCER,XAVIER    138.0   8.0     36.0  20.0
3      BRAZDEIKIS,AUGUSTAS    166.0   8.0    288.0  28.0
4             SIMPSON,NOAH      0.0   0.0    260.0  28.0
5                RILEY,BEN     11.0   0.0      0.0   0.0
6      DOREY-HAVENS,AUBREY      0.0   0.0      0.0   0.0
7         RANDRIASALAMA,TO     97.0   2.0      0.0   0.0
8             LATIFF,WAZIR    236.0  13.0    210.0  25.0
9            MILON,EMANUEL     11.0   0.0      0.0   0.0
10  SERAPHIN,REGINALD JEAN    108.0   5.0      0.0   0.0
11            SMITH,DANIEL      0.0   0.0      0.0   0.0

In [64]:
event_num_df

,player,made layup,missed layup,Assist,Turnover,defensive rebound,enters the game,goes to the bench,missed 3-pt. jump shot,Foul,Steal,made free throw,missed free throw,made jump shot,made 3-pt. jump shot,missed jump shot,offensive rebound
0,"OKADO,MARJOK",2,4,2,1,4,6,6,4,1,0,4,0,1,2,1,0
1,"SIMPSON,NOAH",4,0,1,0,8,2,5,0,3,0,0,0,0,0,0,2
2,"BRAZDEIKIS,AUGUSTAS",4,4,2,2,4,6,7,1,2,1,0,2,0,0,1,1
3,"SPENCER,XAVIER",2,1,5,3,2,5,7,2,4,2,6,1,0,0,1,0
4,"DOREY-HAVENS,AUBREY",3,1,1,2,2,7,4,4,2,0,7,2,0,2,0,0
5,"MILON,EMANUEL",3,1,0,1,3,6,3,0,4,1,3,0,0,0,0,4
6,"LATIFF,WAZIR",1,1,1,0,2,5,5,1,0,0,2,0,0,3,0,0
7,"UGBAH,EMMANUEL",0,1,1,1,3,5,7,0,1,1,0,0,0,0,0,0
8,"RILEY,BEN",0,0,0,1,0,2,2,1,1,0,0,0,0,0,0,0
9,"SERAPHIN,REGINALD JEAN",0,0,0,0,1,4,4,0,5,1,0,0,0,0,0,0


In [65]:
event_num_df5min

player   quarter2                               \
                    player made layup missed layup Assist Turnover   
0             LATIFF,WAZIR          0            1      0        0   
1   SERAPHIN,REGINALD JEAN          0            0      0        0   
2            MILON,EMANUEL          0            1      0        0   
3                RILEY,BEN          0            0      0        0   
4           SPENCER,XAVIER          0            1      0        0   
5      DOREY-HAVENS,AUBREY          0            0      0        0   
6         RANDRIASALAMA,TO          0            0      0        0   
7           UGBAH,EMMANUEL          0            0      0        0   
8             OKADO,MARJOK          0            1      1        0   
9      BRAZDEIKIS,AUGUSTAS          0            0      0        0   
10            SMITH,DANIEL          0            0      0        0   
11            SIMPSON,NOAH          0            0      0        0   

                                                                               \
   defensive rebound enters the game goes to the bench missed 3-pt. jump shot   
0                  0               1                 1                      0   
1                  1               1                 1                      0   
2                  0               1                 1                      0   
3                  0               0                 1                      0   
4                  0               0                 1                      0   
5                  1               1                 0                      0   
6                  0               0                 1                      0   
7                  0               1                 1                      0   
8                  2               2                 1                      1   
9                  0               1                 1                      0   
10                 0               1                 0                      1   
11                 0               0                 0                      0   

         ...          quarter4                                    \
   Foul  ... goes to the bench missed 3-pt. jump shot Foul Steal   
0     0  ...                 1                      0    0     0   
1     2  ...                 0                      0    0     0   
2     0  ...                 0                      0    0     0   
3     0  ...                 0                      0    0     0   
4     1  ...                 1                      1    1     0   
5     1  ...                 0                      0    0     0   
6     0  ...                 0                      0    0     0   
7     0  ...                 1                      0    0     0   
8     0  ...                 1                      0    0     0   
9     0  ...                 2                      0    0     0   
10    0  ...                 0                      0    0     0   
11    0  ...                 1                      0    0     0   

                                                                          \
   made free throw missed free throw made jump shot made 3-pt. jump shot   
0                2                 0              0                    0   
1                0                 0              0                    0   
2                0                 0              0                    0   
3                0                 0              0                    0   
4                1                 0              0                    0   
5                5                 1              0                    1   
6                0                 0              0                    0   
7                0                 0              0                    0   
8                0                 0              0                    0   
9                0                 0              0                    0   
10               0                 0   

In [24]:
lineup_event_df.iloc[:-1] # starter lineup will be added automatically at end of the match so we ignore it cause no event will happen there(try proving it by removing .iloc)

,lineup,defensive rebound,made layup,Assist,missed jump shot,Foul,Turnover,made free throw,enters the game,goes to the bench,missed 3-pt. jump shot,Steal,offensive rebound,missed layup,made 3-pt. jump shot,made jump shot,missed free throw
0,"[BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]",2.0,3.0,2.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, UGBAH,EMMANUEL]",1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, UGBAH,EMMANUEL]",3.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"[DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, RILEY,BEN, SERAPHIN,REGINALD JEAN]",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"[LATIFF,WAZIR, MILON,EMANUEL, RILEY,BEN, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER]",0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
5,"[BRAZDEIKIS,AUGUSTAS, RANDRIASALAMA,TO, RILEY,BEN, SIMPSON,NOAH, SPENCER,XAVIER]",1.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"[BRAZDEIKIS,AUGUSTAS, RANDRIASALAMA,TO, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]",1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
7,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, UGBAH,EMMANUEL]",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"[BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, UGBAH,EMMANUEL]",2.0,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0


In [25]:
lineup_time_score_df

lineup  \
                                                                                              lineup   
0                  [BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]   
1            [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, UGBAH,EMMANUEL]   
2            [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, UGBAH,EMMANUEL]   
3              [DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, RILEY,BEN, SERAPHIN,REGINALD JEAN]   
4                   [LATIFF,WAZIR, MILON,EMANUEL, RILEY,BEN, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER]   
5                   [BRAZDEIKIS,AUGUSTAS, RANDRIASALAMA,TO, RILEY,BEN, SIMPSON,NOAH, SPENCER,XAVIER]   
6              [BRAZDEIKIS,AUGUSTAS, RANDRIASALAMA,TO, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]   
7             [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, UGBAH,EMMANUEL]   
8                  [BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]   
9             [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, UGBAH,EMMANUEL]   
10           [DOREY-HAVENS,AUBREY, OKADO,MARJOK, SERAPHIN,REGINALD JEAN, SIMPSON,NOAH, SMITH,DANIEL]   
11                [LATIFF,WAZIR, SERAPHIN,REGINALD JEAN, SIMPSON,NOAH, SMITH,DANIEL, SPENCER,XAVIER]   
12                   [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, SIMPSON,NOAH, SMITH,DANIEL, SPENCER,XAVIER]   
13                      [BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, RILEY,BEN, SIMPSON,NOAH, SPENCER,XAVIER]   
14                  [BRAZDEIKIS,AUGUSTAS, RANDRIASALAMA,TO, RILEY,BEN, SIMPSON,NOAH, SPENCER,XAVIER]   
15                 [BRAZDEIKIS,AUGUSTAS, MILON,EMANUEL, RANDRIASALAMA,TO, RILEY,BEN, SPENCER,XAVIER]   
16     [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, RANDRIASALAMA,TO, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER]   
17  [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER]   
18           [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, SPENCER,XAVIER]   
19             [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, OKADO,MARJOK]   
20                  [DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, OKADO,MARJOK, UGBAH,EMMANUEL]   
21                  [DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, SMITH,DANIEL, UGBAH,EMMANUEL]   
22             [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, SMITH,DANIEL]   
23                 [BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]   
24                [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, MILON,EMANUEL, SPENCER,XAVIER, UGBAH,EMMANUEL]   
25        [DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, SERAPHIN,REGINALD JEAN, UGBAH,EMMANUEL]   
26      [DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, RANDRIASALAMA,TO, SERAPHIN,REGINALD JEAN]   
27           [MILON,EMANUEL, OKADO,MARJOK, RANDRIASALAMA,TO, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER]   
28              [BRAZDEIKIS,AUGUSTAS, MILON,EMANUEL, OKADO,MARJOK, RANDRIASALAMA,TO, SPENCER,XAVIER]   
29                 [BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]   
30           [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, SPENCER,XAVIER]   
31                  [DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, OKADO,MARJOK, UGBAH,EMMANUEL]   
32                        [LATIFF,WAZIR, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]   
33                   [BRAZDEIKIS,AUGUSTAS, LATIFF,WAZIR, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER]   
34              [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, OKADO,MARJOK, SIMPSON,NOAH]   
35            [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, SIMPSON,NOAH, SPENCER,XAVIER]   
36            [BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER]   
37                [DOREY-HAVENS,AUBREY

In [26]:
event_num_df

,player,defensive rebound,made layup,Assist,missed jump shot,Foul,Turnover,made free throw,enters the game,goes to the bench,missed 3-pt. jump shot,Steal,offensive rebound,missed layup,made 3-pt. jump shot,made jump shot,missed free throw
0,"OKADO,MARJOK",4,2.0,2.0,1.0,1.0,1.0,4.0,6.0,6.0,4.0,0.0,0.0,4.0,2.0,1.0,0.0
1,"SIMPSON,NOAH",8,4.0,1.0,0.0,3.0,0.0,0.0,2.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,"BRAZDEIKIS,AUGUSTAS",4,4.0,2.0,1.0,2.0,2.0,0.0,6.0,7.0,1.0,1.0,1.0,4.0,0.0,0.0,2.0
3,"SPENCER,XAVIER",2,2.0,5.0,1.0,4.0,3.0,6.0,5.0,7.0,2.0,2.0,0.0,1.0,0.0,0.0,1.0
4,"DOREY-HAVENS,AUBREY",2,3.0,1.0,0.0,2.0,2.0,7.0,7.0,4.0,4.0,0.0,0.0,1.0,2.0,0.0,2.0
5,"MILON,EMANUEL",3,3.0,0.0,0.0,4.0,1.0,3.0,6.0,3.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0
6,"LATIFF,WAZIR",2,1.0,1.0,0.0,0.0,0.0,2.0,5.0,5.0,1.0,0.0,0.0,1.0,3.0,0.0,0.0
7,"UGBAH,EMMANUEL",3,0.0,1.0,0.0,1.0,1.0,0.0,5.0,7.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
8,"RILEY,BEN",0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"SERAPHIN,REGINALD JEAN",1,0.0,0.0,0.0,5.0,0.0,0.0,4.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [27]:
for index, row in lineup_event_df.iterrows():
    if index < 31:
        time_lineup = lineup_time_score_df.iloc[index]["lineup"].to_list()[0]
    else:
        time_lineup = []
    event_lineup = lineup_event_df.iloc[index]["lineup"]
    if time_lineup != event_lineup:
        print(event_lineup, time_lineup, index)

['DOREY-HAVENS,AUBREY', 'LATIFF,WAZIR', 'MILON,EMANUEL', 'OKADO,MARJOK', 'UGBAH,EMMANUEL'] [] 31
['LATIFF,WAZIR', 'OKADO,MARJOK', 'SIMPSON,NOAH', 'SPENCER,XAVIER', 'UGBAH,EMMANUEL'] [] 32
['BRAZDEIKIS,AUGUSTAS', 'LATIFF,WAZIR', 'OKADO,MARJOK', 'SIMPSON,NOAH', 'SPENCER,XAVIER'] [] 33
['BRAZDEIKIS,AUGUSTAS', 'DOREY-HAVENS,AUBREY', 'LATIFF,WAZIR', 'OKADO,MARJOK', 'SIMPSON,NOAH'] [] 34
['BRAZDEIKIS,AUGUSTAS', 'DOREY-HAVENS,AUBREY', 'LATIFF,WAZIR', 'SIMPSON,NOAH', 'SPENCER,XAVIER'] [] 35
['BRAZDEIKIS,AUGUSTAS', 'DOREY-HAVENS,AUBREY', 'OKADO,MARJOK', 'SIMPSON,NOAH', 'SPENCER,XAVIER'] [] 36
['DOREY-HAVENS,AUBREY', 'MILON,EMANUEL', 'OKADO,MARJOK', 'SPENCER,XAVIER', 'UGBAH,EMMANUEL'] [] 37
['BRAZDEIKIS,AUGUSTAS', 'DOREY-HAVENS,AUBREY', 'MILON,EMANUEL', 'OKADO,MARJOK', 'SPENCER,XAVIER'] [] 38
['DOREY-HAVENS,AUBREY', 'MILON,EMANUEL', 'OKADO,MARJOK', 'SPENCER,XAVIER', 'UGBAH,EMMANUEL'] [] 39
['BRAZDEIKIS,AUGUSTAS', 'OKADO,MARJOK', 'SIMPSON,NOAH', 'SPENCER,XAVIER', 'UGBAH,EMMANUEL'] [] 40


off possession = made and miss

def possession = Turnover, defensive rebound,	Foul, Steal

efficiency = point, rebound, assist, steal, block, -missed free throw, -turnover, -points conceded

What type of game was it? (Pre-season, Regular Season, or Playoffs)

In [28]:
scoring_values = {'made layup': 2, 'made free throw': 1, 'made jump shot': 2, 'made 3-pt. jump shot': 3}
final_columns = ['Player Name', 'PtScored', 'ptsconceded', 
                #  'off possession', 'def possession', 
                "efficiency",
                'minutes', 'home/visitor', 'opponent', 'date', 'game_type']
player_final_table = pd.DataFrame(columns=[final_columns])
for index, row in event_num_df.iterrows():
    
    points_scored = 0
    for key in scoring_values:
        score_value = scoring_values[key]
        points_scored += event_num_df.loc[index, key] * score_value

    points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])

    efficiency = sum([points_scored, row["offensive rebound"], row["defensive rebound"], row["Assist"], row["Steal"]]) - \
    sum([row["missed free throw"], row["Turnover"], points_conceded])
    efficiency = float(efficiency)

    minutes = time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "time")] / 60
    minutes = "{:.2f}".format(minutes.to_list()[0])
    hv_df = player_event_df.loc[player_event_df["player"] == row["player"]].iloc[1]
    if pd.isna(hv_df['Home']) == False:
        hv = "Home"
    else:
        hv = "Visitor"
    
    opponent_df = player_event_df.loc[pd.isna(player_event_df[hv]) == True]
    opponent = opponent_df.iloc[1]["Home"] if hv == "Visitor" else opponent_df.iloc[1]["Visitor"]

    new_row = {"Player Name":[row["player"]], "PtScored":[points_scored],
               "ptsconceded":[points_conceded], "efficiency":[efficiency], "minutes":minutes, 
               "home/visitor":hv, "opponent":opponent}
    
    new_df = pd.DataFrame(new_row)
    player_final_table = pd.concat([player_final_table, new_df], ignore_index=True, axis=0)

player_final_table = player_final_table.reindex(columns=final_columns)

/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_20650/1449936464.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])
/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_20650/1449936464.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])


In [29]:
player_final_table

,Player Name,PtScored,ptsconceded,efficiency,minutes,home/visitor,opponent,date,game_type
0,"OKADO,MARJOK",16.0,51.0,-30.0,26.08,Visitor,Guelph,NaN,NaN
1,"SIMPSON,NOAH",8.0,47.0,-28.0,21.65,Visitor,Guelph,NaN,NaN
2,"BRAZDEIKIS,AUGUSTAS",8.0,59.0,-47.0,28.65,Visitor,Guelph,NaN,NaN
3,"SPENCER,XAVIER",10.0,65.0,-50.0,24.30,Visitor,Guelph,NaN,NaN
4,"DOREY-HAVENS,AUBREY",19.0,36.0,-18.0,19.45,Visitor,Guelph,NaN,NaN
5,"MILON,EMANUEL",9.0,32.0,-16.0,16.55,Visitor,Guelph,NaN,NaN
6,"LATIFF,WAZIR",13.0,35.0,-19.0,18.03,Visitor,Guelph,NaN,NaN
7,"UGBAH,EMMANUEL",0.0,37.0,-33.0,19.72,Visitor,Guelph,NaN,NaN
8,"RILEY,BEN",0.0,9.0,-10.0,4.20,Visitor,Guelph,NaN,NaN
9,"SERAPHIN,REGINALD JEAN",0.0,17.0,-15.0,8.05,Visitor,Guelph,NaN,NaN


## Let's go to lineup

In [30]:
if "Player Name" in final_columns:
    final_columns.remove("Player Name")
lineup_final_columns = ["Lineup"].extend(final_columns)
lineup_final_table = pd.DataFrame(columns=[lineup_final_columns])
for index, row in lineup_event_df.iloc[:-1].iterrows():
    
    points_scored = 0
    for key in scoring_values:
        score_value = scoring_values[key]
        points_scored += lineup_event_df.loc[index, key] * score_value

    points_conceded = float(lineup_time_score_df[("total", "score")].iloc[index])

    efficiency = sum([points_scored, row["offensive rebound"], row["defensive rebound"], row["Assist"], row["Steal"]]) - \
    sum([row["missed free throw"], row["Turnover"], points_conceded])
    efficiency = float(efficiency)

    minutes = lineup_time_score_df[("total", "time")].iloc[index] / 60
    minutes = "{:.2f}".format(minutes)

    hv_df = player_event_df.loc[player_event_df["player"] == row["lineup"][0]].iloc[1]
    if pd.isna(hv_df['Home']) == False:
        hv = "Home"
    else:
        hv = "Visitor"

    opponent_df = player_event_df.loc[pd.isna(player_event_df[hv]) == True]
    opponent = opponent_df.iloc[1]["Home"] if hv == "Visitor" else opponent_df.iloc[1]["Visitor"]

    new_row = {"Lineup":[row["lineup"]], "PtScored":[points_scored],
               "ptsconceded":[points_conceded], "efficiency":[efficiency], 
               "minutes":minutes, "home/visitor":hv, "opponent":opponent}
    
    new_df = pd.DataFrame(new_row)
    lineup_final_table = pd.concat([lineup_final_table, new_df], ignore_index=True, axis=0)

lineup_final_table = lineup_final_table.reindex(columns=lineup_final_columns)

In [31]:
lineup_final_table

,None,Lineup,PtScored,ptsconceded,efficiency,minutes,home/visitor,opponent
0,NaN,"[BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]",8.0,7.0,4.0,3.48,Visitor,Guelph
1,NaN,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, MILON,EMANUEL, OKADO,MARJOK, UGBAH,EMMANUEL]",2.0,0.0,3.0,0.42,Visitor,Guelph
2,NaN,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, UGBAH,EMMANUEL]",5.0,0.0,9.0,1.60,Visitor,Guelph
3,NaN,"[DOREY-HAVENS,AUBREY, LATIFF,WAZIR, MILON,EMANUEL, RILEY,BEN, SERAPHIN,REGINALD JEAN]",0.0,2.0,-2.0,0.52,Visitor,Guelph
4,NaN,"[LATIFF,WAZIR, MILON,EMANUEL, RILEY,BEN, SERAPHIN,REGINALD JEAN, SPENCER,XAVIER]",2.0,1.0,3.0,1.18,Visitor,Guelph
5,NaN,"[BRAZDEIKIS,AUGUSTAS, RANDRIASALAMA,TO, RILEY,BEN, SIMPSON,NOAH, SPENCER,XAVIER]",4.0,4.0,0.0,1.45,Visitor,Guelph
6,NaN,"[BRAZDEIKIS,AUGUSTAS, RANDRIASALAMA,TO, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]",0.0,2.0,-1.0,1.05,Visitor,Guelph
7,NaN,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, UGBAH,EMMANUEL]",2.0,0.0,2.0,0.60,Visitor,Guelph
8,NaN,"[BRAZDEIKIS,AUGUSTAS, OKADO,MARJOK, SIMPSON,NOAH, SPENCER,XAVIER, UGBAH,EMMANUEL]",0.0,0.0,0.0,0.17,Visitor,Guelph
9,NaN,"[BRAZDEIKIS,AUGUSTAS, DOREY-HAVENS,AUBREY, OKADO,MARJOK, SIMPSON,NOAH, UGBAH,EMMANUEL]",7.0,3.0,8.0,1.97,Visitor,Guelph


In [37]:
import pandas as pd

# Create a sample dataframe
data = {'A': [1, 2, 3], 'B': [4, 5, 6], "C":[4, 5, 6]}
df = pd.DataFrame(data)

# Create multi-level columns
df.columns = pd.MultiIndex.from_tuples([('Column 1', 'time'), ('Column 1', 'score'), ('Column 2', 'time')])

# Display the dataframe
print(df)

  Column 1       Column 2
      time score     time
0        1     4        4
1        2     5        5
2        3     6        6
